## Weaviate Query Agent with DSPy (Simple)

This notebook will show you how to define the Weaviate Query Agent as a tool with DSPy.

### Requirements
1. Weaviate Cloud instance (WCD): The Weaviate Query Agent is only accessible through WCD at the moment. You can create a serverless cluster or a free 14-day sandbox [here](https://console.weaviate.cloud/).
1. Install DSPy with `pip install -U dspy`
1. Install the Weaviate Agents package with `pip install weaviate-agents`
1. You'll need a Weaviate cluster with data. If you don't have one, check out [this notebook](integrations/Weaviate-Import-Example.ipynb) to import the Weaviate Blogs.


### Import libraries and keys

In [ ]:
import weaviate
from weaviate_agents.query import QueryAgent
import os
import logging

import dspy

In [ ]:
os.environ["WEAVIATE_URL"] = ""
os.environ["WEAVIATE_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

### Define Query Agent function

In [ ]:
def query_agent_request(query: str) -> str:
    """
    Send a query to the database and get the response.

    Args:
        query (str): The question or query to search for in the database. This can be any natural language question related to the content stored in the database.

    Returns:
        str: The response from the database containing relevant information.
    """

    # connect to your Weaviate Cloud instance
    weaviate_client = weaviate.connect_to_weaviate_cloud(
        cluster_url=os.getenv("WEAVIATE_URL"), 
        auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
        headers={ # add the API key to the model provider from your Weaviate collection, for example `headers={"X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")}` 
        }
    )

    # connect the query agent to your Weaviate collection(s)
    query_agent = QueryAgent(
        client=weaviate_client,
        collections=["Blogs"] 
    )
    return query_agent.run(query).final_answer

In [5]:
# Only show warnings and errors from LiteLLM
logging.getLogger("LiteLLM").setLevel(logging.WARNING)


### Configure LLM

In [ ]:
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))
dspy.configure(lm=lm)

### Create agent

In [ ]:
react_with_query_agent_signature = dspy.Signature("user_question: str -> helpful_response: str")
react_with_query_agent = dspy.ReAct(react_with_query_agent_signature, tools=[query_agent_request], max_iters=2)

### Query time

In [8]:
print(react_with_query_agent.forward(user_question="How do I run Weaviate with Docker?"))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/weaviate/warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


Prediction(
    trajectory={'thought_0': 'I need to find specific instructions on how to run Weaviate with Docker, including any necessary commands or configurations.', 'tool_name_0': 'send_query_agent_request', 'tool_args_0': {'query': 'How to run Weaviate with Docker?'}, 'observation_0': "Failed to execute: {'error': {'message': 'Error in evaluate node', 'code': 'QUERY_AGENT_NODE_ERROR', 'details': {'node_type': 'evaluate'}}}", 'thought_1': 'Since the query to the database failed, I will try to find alternative resources or documentation on running Weaviate with Docker.', 'tool_name_1': 'send_query_agent_request', 'tool_args_1': {'query': 'Weaviate Docker installation guide'}, 'observation_1': QueryAgentResponse(original_query='Weaviate Docker installation guide', collection_names=['Blogs'], searches=[[QueryResultWithCollection(queries=['Weaviate Docker installation guide'], filters=[[]], filter_operators='AND', collection='Blogs')]], aggregations=[], usage=Usage(requests=3, request_